In [ ]:
import importlib
import subprocess
import sys
import gc

def check_and_install_package(package_name, version=None):
    try:
        importlib.import_module(package_name)
        print(f"\n{package_name} is already installed.")
    except ImportError:
        print(f"\n{package_name} is NOT installed. Installing now...")
        if version:
            subprocess.check_call([sys.executable, "-m", "pip", "install", f"{package_name}=={version}"])
        else:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"{package_name} installation completed.")

# List of packages to check along with specific versions if necessary
packages = [
    {"name": "tqdm", "version": None},
    {"name": "gdown", "version": None},
    {"name": "numpy", "version": "1.23.5"},
    {"name": "pandas", "version": None}  # Added pandas
]

# Checking and installing packages
for package in packages:
    check_and_install_package(package["name"], package["version"])



tqdm is already installed.

gdown is already installed.

numpy is already installed.

pandas is already installed.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

!cp '/content/drive/MyDrive/Big Data Analytics - Project/Datasets/Feature_Engineered_DF.parquet' /content/

# Loading the parquet file into a pandas DataFrame
output_path = '/content/Feature_Engineered_DF.parquet'
df = pd.read_parquet(output_path)

print("The Feature Engineered DataFrame has been loaded successfully.")


The Feature Engineered DataFrame has been loaded successfully.


In [ ]:
df.head()

,fuel_type,body_type,city,city_fuel_economy,days_in_market,dealer_zip,engine_displacement,engine_type,exterior_color,franchise_dealer,...,major_options_count,hp_x_engine_disp,hp_x_torque,listed_day,listed_month,listed_year,age,resale_value_score,maintenance_cost,luxury_score
0,Gasoline,SUV / Crossover,Ontario,20.000000,23,91761,3500.0,V6,Blue,True,...,5,0.18,-0.02369,20,8,2020,0,32,41,33
1,Gasoline,Sedan,Elizabeth,22.000000,22,07202,2000.0,I4,Black,False,...,8,-0.04,0.00000,18,8,2020,3,26,38,36
2,Biodiesel,Pickup Truck,Omaha,22.690001,93,68134,6700.0,V8,White,True,...,5,7.40,0.00003,9,6,2020,0,30,46,34
3,Gasoline,SUV / Crossover,Clearwater,22.690001,163,33763,1500.0,I4,Black,True,...,5,0.97,0.50512,31,3,2020,0,30,37,32
4,Gasoline,SUV / Crossover,Chillicothe,18.000000,25,64601,3000.0,V6,Blue,True,...,15,0.08,0.00002,16,8,2020,0,28,45,39


In [ ]:
import pandas as pd
import numpy as np
import time
import warnings
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Ignore warnings
warnings.filterwarnings('ignore')

# Starting to track overall runtime
start_time = time.time()

# Combining processing data and model training in the same progress bar
with tqdm(total=6, desc="Processing and Training") as pbar:

    # Sampling 40% of the data
    df_sample = df.sample(frac=0.4, random_state=42)
    pbar.update(1)

    # Handling categorical columns
    cat_columns = df_sample.select_dtypes(include=['object']).columns.tolist()
    num_columns = df_sample.select_dtypes(exclude=['object']).columns.tolist()
    num_columns.remove('price')
    pbar.update(1)

    # Converting 'franchise_dealer' to numeric
    df_sample['franchise_dealer'] = df_sample['franchise_dealer'].astype(int)

    # Preprocessing pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_columns),
            ('cat', OneHotEncoder(handle_unknown='ignore'), cat_columns)
        ]
    )
    pbar.update(1)

    # Splitting the data into training and test sets
    X = df_sample.drop(columns='price')
    y = df_sample['price']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    pbar.update(1)

    # Applying the preprocessing pipeline
    X_train_transformed = preprocessor.fit_transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    pbar.update(1)

# Training and evaluating for different degrees of Polynomial Regression

for degree in range(1, 2):  # Iterating over degrees 1, 2
    print(f"\nTraining Polynomial Regression with degree={degree}...")

    # Applying PolynomialFeatures to introduce polynomial combinations of the features
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_train_poly = poly.fit_transform(X_train_transformed)
    X_test_poly = poly.transform(X_test_transformed)

    # Training Polynomial Regression model (Linear Regression with polynomial features)
    lr = LinearRegression()
    model = lr.fit(X_train_poly, y_train)

    # Making predictions
    print(f"Making predictions for degree={degree}...")
    y_pred = model.predict(X_test_poly)

    # Evaluating the model
    r2 = r2_score(y_test, y_pred)

    # Displaying results for the current degree
    print(f"\nResults for degree={degree}:")
    print(f"Train size: {len(X_train):,} samples")
    print(f"Test size: {len(X_test):,} samples")
    print(f"R-Squared Score (Accuracy): {r2 * 100:.2f}%")

    # Calculating additional metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    # Outputing additional metrics
    print("Additional Metrics:")
    print(f"Mean Absolute Error: {round(mae, 2)}")
    print(f"Mean Squared Error: {round(mse, 2)}")
    print(f"Root Mean Squared Error: {round(rmse, 2)}")
    print("--------------------------------------------------------------------------------------------------")

# Calculating total runtime
end_time = time.time()
total_runtime = (end_time - start_time) / 60  # Convert seconds to minutes
print(f"\nOverall runtime: {round(total_runtime)} minutes.")


Processing and Training:  83%|████████▎ | 5/6 [00:11<00:02,  2.27s/it]



Training Polynomial Regression with degree=1...
Making predictions for degree=1...

Results for degree=1:
Train size: 960,012 samples
Test size: 240,004 samples
R-Squared Score (Accuracy): 81.87%
Additional Metrics:
Mean Absolute Error: 4058.64
Mean Squared Error: 65629206.94
Root Mean Squared Error: 8101.19
--------------------------------------------------------------------------------------------------

Training Polynomial Regression with degree=2...
